<a href="https://colab.research.google.com/github/gitkuntal/colab-notebook/blob/master/exp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello world!!")

Hello world!!


In [ ]:
# Install OpenAI package (if not already installed)
!pip install --quiet openai
!pip install openai chromadb
!pip install -U langchain-community
!pip install langchain-openai

In [ ]:

# Import required libraries
import getpass
from openai import OpenAI

# Ask for API key securely (you only need to do this once per Colab session)
api_key = getpass.getpass("🔑 Enter your OpenAI API Key: ")
#api_key = ""

# Initialize client
client = OpenAI(api_key=api_key)

In [2]:
# Import required libraries
from openai import OpenAI


# Ask user for a query
#query = input("📝 Enter your query for GPT-5: ")

query = "tell me how can I edit image using nano banana."

# Send request to GPT-5
response = client.chat.completions.create(
    model="gpt-4o-mini",  # using gpt-4o-mini
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": query}
    ]
)

# Print response
print("\n=== LLM Response ===\n")
print(response.choices[0].message.content)


=== LLM Response ===

"Nano banana" is likely a humorous or playful term referring to the image editing software "GIMP" (GNU Image Manipulation Program) due to its logo, which features a character that resembles a cartoon banana. If you meant a specific tool or library, please clarify. However, I'll provide some general guidance on how to edit images using GIMP.

### How to Edit Images Using GIMP

1. **Install GIMP**:
   - Download GIMP from [the official GIMP website](https://www.gimp.org/downloads/).
   - Follow the installation instructions for your operating system.

2. **Open GIMP**:
   - Launch GIMP after it’s installed.

3. **Open an Image**:
   - Go to `File` > `Open` or simply drag and drop your image file into the GIMP window.

4. **Basic Editing Tools**:
   - **Selection Tools**: Use the selection tools (rectangle, ellipse, free select, fuzzy select) to select parts of the image you want to edit.
   - **Brush Tool**: Use the paintbrush tool to draw or paint on the image.
  

In [32]:

import json
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


In [33]:
# Load the JSON
with open("search_data.json", "r", encoding="utf-8") as f:
    search_docs = json.load(f)

# Extract texts from search_pages
search_texts = [f"{item['title']} - {item['snippet']}" for item in search_docs['search_pages']]

# Extract texts from pubmed_pages
pubmed_texts = [f"{item['pub_title']} - {item['abstract_snippet']}" for item in search_docs['pubmed_pages']]

In [40]:
# 2. Create embedding function wrapper for Chroma
embeddings_client = OpenAIEmbeddings(
    api_key=api_key,
    model="text-embedding-3-small"
)

In [41]:

# Build Chroma collection from search_pages
web_search_db = Chroma.from_texts(
    search_texts,
    embedding=embeddings_client,
    collection_name="web_search_db_demo2"
)

# Build another collection from pubmed_pages (optional)
pubmed_db = Chroma.from_texts(
    pubmed_texts,
    embedding=embeddings_client,
    collection_name="pubmed_db_demo2"
)

In [43]:
query = "What are the complications of diabetes?"
results = web_search_db.similarity_search(query, k=2)

for i, res in enumerate(results, 1):
    print(f"{i}. {res.page_content}\n")

1. Diabetes mellitus: The epidemic of the century - Diabetes mellitus is a group of metabolic diseases characterized by chronic hyperglycemia resulting from defects in insulin secretion, insulin action, or both.

2. Diabetes Care (journal website) - Diabetes Care publishes original research about the physiology and pathophysiology of diabetes mellitus. Submitted manuscripts can report any aspect of laboratory, clinical, epidemiologic, health services, translational research, etc.



In [42]:
query = "What are the complications of diabetes?"
results = pubmed_db.similarity_search(query, k=2)

for i, res in enumerate(results, 1):
    print(f"{i}. {res.page_content}\n")


1. The burden and risks of emerging complications associated with diabetes mellitus - This article discusses evidence for the emergence of a different set of complications associated with diabetes mellitus from the traditional ones. These include increased risks of non‐vascular complications such as cancer, infections, neurodegenerative diseases, and liver disease in patients with long‐standing hyperglycemia.

2. Diabetes mellitus: Classification, mediators, and therapeutic challenges - Diabetes mellitus is defined by persistent hyperglycemia and is commonly classified into type 1, type 2, gestational diabetes and other specific types. Key mediators include insulin resistance, β-cell dysfunction, inflammation, and lipid dysregulation. Managing these factors remains central to therapy.

